In [1]:
#import statements
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore') 

weight1=[[ 0.1642,  0.2455, -0.3093, -0.4260, -0.1927],
        [ 0.1405,  0.2877,  0.1791,  0.2886,  0.0281],
        [-0.4129,  0.2434, -0.3342,  0.0638,  0.0898],
        [ 0.3933,  0.3019,  0.0625,  0.2342, -0.4222],
        [-0.3019,  0.1200, -0.1640, -0.1984, -0.1032]]

weight2=[[1.0,1.0,1.0,1.0,1.0],[1.0,1.0,1.0,1.0,1.0],[1.0,1.0,1.0,1.0,1.0],[1.0,1.0,1.0,1.0,1.0],[1.0,1.0,1.0,1.0,1.0]]

weight4=[[-0.0068,  0.1320, -0.4003,  0.0965, -0.4061],
        [ 0.2241,  0.1185,  0.0251, -0.0104, -0.0224],
        [ 0.4361, -0.0875,  0.0647,  0.3484,  0.4034],
        [ 0.1191, -0.3272,  0.2531,  0.2740, -0.3526],
        [ 0.3558, -0.0614, -0.2608, -0.1395, -0.0635]]


weight5=[[-0.0115,  0.1046,  0.2259,  0.1117, -0.0206],
        [-0.4093, -0.2933, -0.3829, -0.2851,  0.1360],
        [ 0.1195,  0.3427,  0.4183,  0.1673,  0.1913],
        [-0.2937, -0.2824,  0.1436, -0.2029,  0.1594],
        [ 0.0140,  0.1943, -0.2809,  0.2599,  0.0647]]

weightout=[[.50,.50,.50,.50,.50]]
bias1=[ 0.1889, -0.4299,  0.0239,  0.3833, -0.2821]
bias2=[0.0,.0,0.0,0.0,0.0]
#weight2
#bias3=
#bias4=
#bias5=


In [2]:
#Data class
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)


In [3]:
#model class
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        # Number of input features is 12.
        self.layer_1 = nn.Linear(5, 5) #layer 1
        self.layer_2 = nn.Linear(5, 5) #layer 2 input matches output of prev layer
        self.layer_3 = nn.Linear(5, 5)
        self.layer_4= nn.Linear(5,5)
        self.layer_5=nn.Linear(5,5)
        self.layer_out = nn.Linear(5, 1) #output layer
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(5)
        self.batchnorm2 = nn.BatchNorm1d(5)
        self.batchnorm3 = nn.BatchNorm1d(5)
        self.batchnorm4= nn.BatchNorm1d(5)
        self.batchnorm5= nn.BatchNorm1d(5)

        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.relu(self.layer_4(x))
        x = self.batchnorm4(x)
        x = self.relu(self.layer_5(x))
        x = self.batchnorm5(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [4]:
#Data loaders (to look at data) and deal with data (training and testing)
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)



In [5]:
#accuracy calculatory rounds output to 0 or 1
def binary_acc(y_pred, y_test): 
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [6]:
#groundtruth model
df = pd.read_csv("Breast_cancer_data.csv")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")




X = df.iloc[:, 0:-1] #input columns
scaler = StandardScaler()
X_allscaled = scaler.fit_transform(X) #scales data
X_formatted = testData(torch.FloatTensor(X_allscaled)) #format data for input
Truth_loader = DataLoader(dataset=X_formatted, batch_size=1) #format data for input




GroundTruth = binaryClassification()
#set weights to "evenly distributed" 0, 1
GroundTruth.layer_1.weight=torch.nn.Parameter(data=torch.tensor(weight1), requires_grad=True)
GroundTruth.layer_2.weight=torch.nn.Parameter(data=torch.tensor(weight2), requires_grad=True)
GroundTruth.layer_3.weight=torch.nn.Parameter(data=torch.tensor(weight2), requires_grad=True)
GroundTruth.layer_4.weight=torch.nn.Parameter(data=torch.tensor(weight4), requires_grad=True)
GroundTruth.layer_5.weight=torch.nn.Parameter(data=torch.tensor(weight5), requires_grad=True)
GroundTruth.layer_out.weight=torch.nn.Parameter(data=torch.tensor(weightout), requires_grad=True)

GroundTruth.layer_1.bias=torch.nn.Parameter(data=torch.tensor(bias1), requires_grad=True)
GroundTruth.layer_2.bias=torch.nn.Parameter(data=torch.tensor(bias2), requires_grad=True)
GroundTruth.layer_3.bias=torch.nn.Parameter(data=torch.tensor(bias2), requires_grad=True)
GroundTruth.layer_4.bias=torch.nn.Parameter(data=torch.tensor(bias2), requires_grad=True)
GroundTruth.layer_5.bias=torch.nn.Parameter(data=torch.tensor(bias2), requires_grad=True)
GroundTruth.layer_out.bias=torch.nn.Parameter(data=torch.tensor([-3.0]),requires_grad=True)


#print(GroundTruth.layer_1.weight)


truth_list = []
GroundTruth.eval()
with torch.no_grad():
    for X_batch in Truth_loader:
        X_batch = X_batch.to(device)
        y_truth = GroundTruth(X_batch)
        y_truth = torch.sigmoid(y_truth)
        y_truthtag = torch.round(y_truth)
        truth_list.append(y_truthtag.cpu().numpy())

y = [a.squeeze().tolist() for a in truth_list] #new truth values
#print(y)

#split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=69)
#print(y_train)
#rescale data 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

EPOCHS = 50 #number of passes of whole data
BATCH_SIZE = 64 #size of data going through at once
LEARNING_RATE = 0.001



train_data = trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
## test data    ()

test_data = testData(torch.FloatTensor(X_test))

#data loader initiation
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)




device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [7]:
#train trained model
trained = binaryClassification()
trained.to(device)
#print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(trained.parameters(), lr=LEARNING_RATE)

#print(list(model.parameters())[0])

#train trained model
trained.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = trained(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    #print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')
    
    


In [8]:
#reset untrained model
untrained=binaryClassification()

In [9]:
#prints results
y_trainedpred_list = []
trained.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = trained(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_trainedpred_list.append(y_pred_tag.cpu().numpy())

y_trainedpred_list = [a.squeeze().tolist() for a in y_trainedpred_list]
confusion_matrix(y_test, y_trainedpred_list)
print(classification_report(y_test, y_trainedpred_list))





#untrained results
y_untrainedpred_list = []
untrained.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = untrained(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_untrainedpred_list.append(y_pred_tag.cpu().numpy())

y_untrainedpred_list = [a.squeeze().tolist() for a in y_untrainedpred_list]
confusion_matrix(y_test, y_untrainedpred_list)
print(classification_report(y_test, y_untrainedpred_list))


#print(y_untrainedpred_list)
#print(y_trainedpred_list)

trainedcounter=0
untrainedcounter=0
for i in range(len(y_trainedpred_list)):
    if y_trainedpred_list[i]==y_test[i]:
        trainedcounter=trainedcounter+1       
    if y_untrainedpred_list[i]==y_test[i]:
        untrainedcounter=untrainedcounter+1
        
print("trained correct: ",trainedcounter)
print("untrained correct: ",untrainedcounter)

print("precision=(# of correct)/(# of guessed)")
print("recall=(# of correct)/(total)")

              precision    recall  f1-score   support

         0.0       0.59      0.93      0.72        88
         1.0       0.88      0.43      0.58       100

    accuracy                           0.66       188
   macro avg       0.73      0.68      0.65       188
weighted avg       0.74      0.66      0.65       188

              precision    recall  f1-score   support

         0.0       0.47      1.00      0.64        88
         1.0       0.00      0.00      0.00       100

    accuracy                           0.47       188
   macro avg       0.23      0.50      0.32       188
weighted avg       0.22      0.47      0.30       188

trained correct:  125
untrained correct:  88
precision=(# of correct)/(# of guessed)
recall=(# of correct)/(total)


In [10]:
#Find and fix weights

#layer 1 weights
#print(GroundTruth.layer_1.weight)


#get weighs for seconds layer
#print(repr(GroundTruth.layer_2.weight.detach().numpy()))

#layer 1 weights
#print(GroundTruth.layer_out.weight)


#bais1
#print(GroundTruth.layer_1.bias)


#bais2
#print(GroundTruth.layer_2.bias)


#bais3
#print(GroundTruth.layer_out.bias)



In [13]:
#counts number of groundtruth values
truthcount=0
for i in y:
    if i==1:
        truthcount=truthcount+1
        
#print(truthcount)

In [12]:
#print(len(y_test))
